In [2]:
from pyspark.sql import SparkSession
from typing import List
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Question 1
Given employer history data, where each record contains details about an employee’s work history — employer, job position, and the start and end dates of each job.

We need to find out how many users had Microsoft as their employer, and immediately after that, they started working at Google, with no other employers between these two positions.

In [4]:
# Initialize Spark session
spark = SparkSession.builder.master("local[*]").appName("LinkedInUsers").getOrCreate()

linkedin_data = [
    (1, 'Microsoft', 'developer', '2020-04-13', '2021-11-01'),
    (1, 'Google', 'developer', '2021-11-01', None),
    (2, 'Google', 'manager', '2021-01-01', '2021-01-11'),
    (2, 'Microsoft', 'manager', '2021-01-11', None),
    (3, 'Microsoft', 'analyst', '2019-03-15', '2020-07-24'),
    (3, 'Amazon', 'analyst', '2020-08-01', '2020-11-01'),
    (3, 'Google', 'senior analyst', '2020-11-01', '2021-03-04'),
    (4, 'Google', 'junior developer', '2018-06-01', '2021-11-01'),
    (4, 'Google', 'senior developer', '2021-11-01', None),
    (5, 'Microsoft', 'manager', '2017-09-26', None),
    (6, 'Google', 'CEO', '2015-10-02', None)
]

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/01 14:46:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
linkedin_data_df = spark.createDataFrame(linkedin_data, 
                                         ['user_id', 'company', 'position', 'start_date', 'end_date'])

In [30]:
window_spec = Window.partitionBy('user_id').orderBy(F.col('start_date').asc())

In [31]:
linkedin_data_df = linkedin_data_df.withColumn('next_company', F.lead('company').over(window_spec))

In [32]:
linkedin_data_df.show()

+-------+---------+----------------+----------+----------+------------+
|user_id|  company|        position|start_date|  end_date|next_company|
+-------+---------+----------------+----------+----------+------------+
|      1|Microsoft|       developer|2020-04-13|2021-11-01|      Google|
|      1|   Google|       developer|2021-11-01|      NULL|        NULL|
|      2|   Google|         manager|2021-01-01|2021-01-11|   Microsoft|
|      2|Microsoft|         manager|2021-01-11|      NULL|        NULL|
|      3|Microsoft|         analyst|2019-03-15|2020-07-24|      Amazon|
|      3|   Amazon|         analyst|2020-08-01|2020-11-01|      Google|
|      3|   Google|  senior analyst|2020-11-01|2021-03-04|        NULL|
|      4|   Google|junior developer|2018-06-01|2021-11-01|      Google|
|      4|   Google|senior developer|2021-11-01|      NULL|        NULL|
|      5|Microsoft|         manager|2017-09-26|      NULL|        NULL|
|      6|   Google|             CEO|2015-10-02|      NULL|      

In [37]:
linkedin_data_df.filter((F.col('company') == "Microsoft") &
                        (F.col('next_company') == "Google")).count()


1

# Question 2
Given a table of hotels with various attributes (hotel_address, additional_number_of_scoring, review_date, average_score, hotel_name, reviewer_nationality, negative_review, review_total_negative_word_counts, total_number_of_reviews, positive_review, review_total_positive_word_counts, total_number_of_reviews_reviewer_has_given, reviewer_score, tags, days_since_review, lat, lng ), We need to find the top 10 hotels with the highest average scores. The output should include:

1. The hotel name.
2. The average score of the hotel.
3. The records should be sorted by average score in descending order.

In [39]:
# Initialize Spark session
spark = SparkSession.builder.master("local[*]").appName("TopHotelsReviews").getOrCreate()

25/03/01 15:21:56 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [40]:
# Sample data
data = [
  ('123 Ocean Ave, Miami, FL', 3, '2024-11-10', 4.2, 'Ocean View', 'American', 'Room small, but clean.', 5, 150, 'Great location and friendly staff!', 8, 30, 4.5, 'beachfront, family-friendly', '5 days', 25.7617, -80.1918),   
  ('456 Mountain Rd, Boulder, CO', 2, '2024-11-12', 3.9, 'Mountain Lodge', 'Canadian', 'wifi slow.', 3, 120, 'nice rooms.', 10, 20, 4.0, 'scenic, nature', '3 days', 40.015, -105.2705),  
  ('789 Downtown St, New York, NY', 5, '2024-11-15', 4.7, 'Central Park Hotel', 'British', 'Noisy, sleep.', 7, 200, 'Perfect location near Central Park.', 12, 50, 4.7, 'luxury, city-center', '1 day', 40.7831, -73.9712),
  ('101 Lakeside Blvd, Austin, TX', 1, '2024-11-08', 4.0, 'Lakeside Inn', 'Mexican', 'food avg.', 4, 80, 'Nice, friendly service.', 6, 15, 3.8, 'relaxing, family', '10 days', 30.2672, -97.7431),
  ('202 River Ave, Nashville, TN', 4, '2024-11-13', 4.5, 'Riverside', 'German', 'Limited parking', 2, 175, 'Great rooms.', 9, 25, 4.2, 'riverfront, peaceful', '2 days', 36.1627, -86.7816)
]
# Define columns for the hotel DataFrame
columns = [
  "hotel_address", 
  "additional_number_of_scoring", 
  "review_date", 
  "average_score", 
  "hotel_name",            
  "reviewer_nationality", 
  "negative_review", 
  "review_total_negative_word_counts", 
  "total_number_of_reviews",           
  "positive_review", 
  "review_total_positive_word_counts", 
  "total_number_of_reviews_reviewer_has_given",
  "reviewer_score", 
  "tags", 
  "days_since_review", 
  "lat", 
  "lng"
]

In [41]:
hotel_df = spark.createDataFrame(data, columns)

In [47]:
hotel_df.select("hotel_name", "average_score").orderBy(F.col("average_score").desc()).show(10)

+------------------+-------------+
|        hotel_name|average_score|
+------------------+-------------+
|Central Park Hotel|          4.7|
|         Riverside|          4.5|
|        Ocean View|          4.2|
|      Lakeside Inn|          4.0|
|    Mountain Lodge|          3.9|
+------------------+-------------+



# Question 3
We have a table of employees that includes the following fields: id, first_name, last_name, age, sex, employee_title, department, salary, target, bonus, city, address, and manager_id. We need to find the top 3 distinct salaries for each department. The output should include:

1. The department name.
2. The top 3 distinct salaries for each department.
3. The results should be ordered alphabetically by department and then by the highest salary to the lowest salary.

In [124]:
# Initialize Spark session
spark = SparkSession.builder.appName("TopSalariesByDepartment").getOrCreate()
# Sample data
data = [
    (1, 'Allen', 'Wang', 55, 'F', 'Manager', 'Management', 200000, 0, 300, 'California', '23St', 1),
    (13, 'Katty', 'Bond', 56, 'F', 'Manager', 'Management', 150000, 0, 300, 'Arizona', None, 1),
    (19, 'George', 'Joe', 50, 'M', 'Manager', 'Management', 100000, 0, 300, 'Florida', '26St', 1),
    (11, 'Richerd', 'Gear', 57, 'M', 'Manager', 'Management', 250000, 0, 300, 'Alabama', None, 1),
    (10, 'Jennifer', 'Dion', 34, 'F', 'Sales', 'Sales', 100000, 200, 150, 'Alabama', None, 13),
    (18, 'Laila', 'Mark', 26, 'F', 'Sales', 'Sales', 100000, 200, 150, 'Florida', '23St', 11),
    (20, 'Sarrah', 'Bicky', 31, 'F', 'Senior Sales', 'Sales', 200000, 200, 150, 'Florida', '53St', 19),
    (21, 'Suzan', 'Lee', 34, 'F', 'Sales', 'Sales', 130000, 200, 150, 'Florida', '56St', 19),
    (22, 'Mandy', 'John', 31, 'F', 'Sales', 'Sales', 130000, 200, 150, 'Florida', '45St', 19),
    (17, 'Mick', 'Berry', 44, 'M', 'Senior Sales', 'Sales', 220000, 200, 150, 'Florida', None, 11),
    (12, 'Shandler', 'Bing', 23, 'M', 'Auditor', 'Audit', 110000, 200, 150, 'Arizona', None, 11),
    (14, 'Jason', 'Tom', 23, 'M', 'Auditor', 'Audit', 100000, 200, 150, 'Arizona', None, 11),
    (16, 'Celine', 'Anston', 27, 'F', 'Auditor', 'Audit', 100000, 200, 150, 'Colorado', None, 11),
    (15, 'Michale', 'Jackson', 44, 'F', 'Auditor', 'Audit', 70000, 150, 150, 'Colorado', None, 11),
    (6, 'Molly', 'Sam', 28, 'F', 'Sales', 'Sales', 140000, 100, 150, 'Arizona', '24St', 13),
    (7, 'Nicky', 'Bat', 33, 'F', 'Sales', 'Sales', None, None, None, None, None, None)
]
# Define columns for the employees DataFrame
columns = [
  "id", 
  "first_name", 
  "last_name", 
  "age", 
  "sex", 
  "employee_title", 
  "department", 
  "salary", 
  "target", 
  "bonus", "city", 
  "address", 
  "manager_id"
]

# Create DataFrame
employees = spark.createDataFrame(data, columns)

In [125]:
employees.show()

+---+----------+---------+---+---+--------------+----------+------+------+-----+----------+-------+----------+
| id|first_name|last_name|age|sex|employee_title|department|salary|target|bonus|      city|address|manager_id|
+---+----------+---------+---+---+--------------+----------+------+------+-----+----------+-------+----------+
|  1|     Allen|     Wang| 55|  F|       Manager|Management|200000|     0|  300|California|   23St|         1|
| 13|     Katty|     Bond| 56|  F|       Manager|Management|150000|     0|  300|   Arizona|   NULL|         1|
| 19|    George|      Joe| 50|  M|       Manager|Management|100000|     0|  300|   Florida|   26St|         1|
| 11|   Richerd|     Gear| 57|  M|       Manager|Management|250000|     0|  300|   Alabama|   NULL|         1|
| 10|  Jennifer|     Dion| 34|  F|         Sales|     Sales|100000|   200|  150|   Alabama|   NULL|        13|
| 18|     Laila|     Mark| 26|  F|         Sales|     Sales|100000|   200|  150|   Florida|   23St|        11|
|

In [126]:
window_spec = Window.partitionBy('department').orderBy(F.col('salary').desc())

In [127]:
employees = employees.select("department", "salary").distinct().withColumn('rank', F.dense_rank().over(window_spec))
top_3_salaries = employees.filter((F.col('rank') <= 3))
top_3_salaries.show()

+----------+------+----+
|department|salary|rank|
+----------+------+----+
|     Audit|110000|   1|
|     Audit|100000|   2|
|     Audit| 70000|   3|
|Management|250000|   1|
|Management|200000|   2|
|Management|150000|   3|
|     Sales|220000|   1|
|     Sales|200000|   2|
|     Sales|140000|   3|
+----------+------+----+



In [128]:
top_3_salaries.groupBy('department').agg(F.collect_list("salary")).show(truncate=False)

+----------+------------------------+
|department|collect_list(salary)    |
+----------+------------------------+
|Audit     |[110000, 100000, 70000] |
|Management|[250000, 200000, 150000]|
|Sales     |[220000, 200000, 140000]|
+----------+------------------------+



# Question 4
Given two datasets: one containing signup details (including start and stop times) and another containing transaction details (such as amounts), determine the most profitable location based on signup duration and transaction amounts.

In [158]:
# Initialize Spark session
spark = SparkSession.builder.appName("UberProfitableLocation").getOrCreate()
# Sample Data - Creating the DataFrames for signups and transactions
signups_data = [
    (1, '2020-01-01 10:00:00', '2020-01-01 12:00:00', 101, 'New York'),
    (2, '2020-01-02 11:00:00', '2020-01-02 13:00:00', 102, 'Los Angeles'),
    (3, '2020-01-03 10:00:00', '2020-01-03 14:00:00', 103, 'Chicago'),
    (4, '2020-01-04 09:00:00', '2020-01-04 10:30:00', 101, 'San Francisco'),
    (5, '2020-01-05 08:00:00', '2020-01-05 11:00:00', 102, 'New York')
]
transactions_data = [
    (1, 1, '2020-01-01 10:30:00', 50.00),
    (2, 1, '2020-01-01 11:00:00', 30.00),
    (3, 2, '2020-01-02 11:30:00', 100.00),
    (4, 2, '2020-01-02 12:00:00', 75.00),
    (5, 3, '2020-01-03 10:30:00', 120.00),
    (6, 4, '2020-01-04 09:15:00', 80.00),
    (7, 5, '2020-01-05 08:30:00', 90.00)
]
# Define columns for signups DataFrame
signups_columns = [
  "signup_id", 
  "signup_start_date", 
  "signup_stop_date", 
  "plan_id", 
  "location"
]
signups_df = spark.createDataFrame(signups_data, signups_columns)


# Define columns for transactions DataFrame
transactions_columns = [
  "transaction_id", 
  "signup_id", 
  "transaction_start_date", 
  "amt"
]
transactions_df = spark.createDataFrame(transactions_data, transactions_columns)

In [159]:
signups_df.show()

+---------+-------------------+-------------------+-------+-------------+
|signup_id|  signup_start_date|   signup_stop_date|plan_id|     location|
+---------+-------------------+-------------------+-------+-------------+
|        1|2020-01-01 10:00:00|2020-01-01 12:00:00|    101|     New York|
|        2|2020-01-02 11:00:00|2020-01-02 13:00:00|    102|  Los Angeles|
|        3|2020-01-03 10:00:00|2020-01-03 14:00:00|    103|      Chicago|
|        4|2020-01-04 09:00:00|2020-01-04 10:30:00|    101|San Francisco|
|        5|2020-01-05 08:00:00|2020-01-05 11:00:00|    102|     New York|
+---------+-------------------+-------------------+-------+-------------+



In [160]:
transactions_df.show()

+--------------+---------+----------------------+-----+
|transaction_id|signup_id|transaction_start_date|  amt|
+--------------+---------+----------------------+-----+
|             1|        1|   2020-01-01 10:30:00| 50.0|
|             2|        1|   2020-01-01 11:00:00| 30.0|
|             3|        2|   2020-01-02 11:30:00|100.0|
|             4|        2|   2020-01-02 12:00:00| 75.0|
|             5|        3|   2020-01-03 10:30:00|120.0|
|             6|        4|   2020-01-04 09:15:00| 80.0|
|             7|        5|   2020-01-05 08:30:00| 90.0|
+--------------+---------+----------------------+-----+



In [161]:
signups_df = signups_df.withColumn('duration', 
                                   (F.unix_timestamp('signup_stop_date') - F.unix_timestamp('signup_start_date')) / 60)

In [162]:
transactions_df_groupped = transactions_df.groupBy('signup_id').agg(F.sum('amt').alias('total_amt'))

In [164]:
transactions_aggregated_df = transactions_df_groupped.join(signups_df, 'signup_id').orderBy(F.col('total_amt').desc())

In [169]:
transactions_aggregated_df = transactions_aggregated_df.groupBy("location").agg(F.sum("total_amt").alias("total_amt"), 
                                                                                F.sum("duration").alias("duration"))

In [170]:
transactions_aggregated_df.withColumn('profit_per_min', F.col('total_amt') / F.col('duration')).orderBy(F.col('profit_per_min').desc()).show()

+-------------+---------+--------+------------------+
|     location|total_amt|duration|    profit_per_min|
+-------------+---------+--------+------------------+
|  Los Angeles|    175.0|   120.0|1.4583333333333333|
|San Francisco|     80.0|    90.0|0.8888888888888888|
|     New York|    170.0|   300.0|0.5666666666666667|
|      Chicago|    120.0|   240.0|               0.5|
+-------------+---------+--------+------------------+



# Question 5
The problem is to calculate the minimum number of platforms required at a train station based on the given arrival_times and departure_times.

In [171]:
# Create Spark session
spark = SparkSession.builder.master("local[*]").appName("Train Platform Calculation").getOrCreate()
# Sample Data (Train Arrival and Departure times)
arrivals_data = [
    (1, '2024-11-17 08:00'),
    (2, '2024-11-17 08:05'),
    (3, '2024-11-17 08:05'),
    (4, '2024-11-17 08:10'),
    (5, '2024-11-17 08:10'),
    (6, '2024-11-17 12:15'),
    (7, '2024-11-17 12:20'),
    (8, '2024-11-17 12:25'),
    (9, '2024-11-17 15:00'),
    (10, '2024-11-17 15:00'),
    (11, '2024-11-17 15:00'),
    (12, '2024-11-17 15:06'),
    (13, '2024-11-17 20:00'),
    (14, '2024-11-17 20:10')
]

departures_data = [
    (1, '2024-11-17 08:15'),
    (2, '2024-11-17 08:10'),
    (3, '2024-11-17 08:20'),
    (4, '2024-11-17 08:25'),
    (5, '2024-11-17 08:20'),
    (6, '2024-11-17 13:00'),
    (7, '2024-11-17 12:25'),
    (8, '2024-11-17 12:30'),
    (9, '2024-11-17 15:05'),
    (10, '2024-11-17 15:10'),
    (11, '2024-11-17 15:15'),
    (12, '2024-11-17 15:15'),
    (13, '2024-11-17 20:15'),
    (14, '2024-11-17 20:15')
]
# Define schema for the data
arrival_columns = ['train_id', 'arrival_time']
departure_columns = ['train_id', 'departure_time']
# Create DataFrames
arrivals_df = spark.createDataFrame(arrivals_data, arrival_columns)
departures_df = spark.createDataFrame(departures_data, departure_columns)

25/03/01 17:10:58 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [172]:
# Convert the time strings to timestamps for easier handling
arrivals_df = arrivals_df.withColumn('arrival_time', F.col('arrival_time').cast('timestamp'))
departures_df = departures_df.withColumn('departure_time', F.col('departure_time').cast('timestamp'))

In [184]:
arrivals_window = Window.partitionBy('arrival_time').orderBy(F.col('arrival_time').asc())
departures_window = Window.partitionBy('departure_time').orderBy(F.col('departure_time').asc())

In [195]:
arrivals_groupped_df = arrivals_df.groupBy("arrival_time").agg(F.count("train_id").alias("trains_arrived")).orderBy(F.col("arrival_time").asc())
departures_groupped_df = departures_df.groupBy("departure_time").agg(F.count("train_id").alias("trains_departed")).orderBy(F.col("departure_time").asc())

In [212]:
schedule_df = arrivals_groupped_df.join(departures_groupped_df,
                          (arrivals_groupped_df["arrival_time"] == departures_groupped_df["departure_time"]),
                          'outer').fillna(0)

schedule_df = schedule_df.select(F.coalesce("arrival_time","departure_time").alias("time"),
                   "trains_arrived",
                   "trains_departed",
                   (F.col("trains_arrived") - F.col("trains_departed")).alias("difference"))

In [213]:
schedule_df.show()

+-------------------+--------------+---------------+----------+
|               time|trains_arrived|trains_departed|difference|
+-------------------+--------------+---------------+----------+
|2024-11-17 08:00:00|             1|              0|         1|
|2024-11-17 08:05:00|             2|              0|         2|
|2024-11-17 08:10:00|             2|              1|         1|
|2024-11-17 08:15:00|             0|              1|        -1|
|2024-11-17 08:20:00|             0|              2|        -2|
|2024-11-17 08:25:00|             0|              1|        -1|
|2024-11-17 12:15:00|             1|              0|         1|
|2024-11-17 12:20:00|             1|              0|         1|
|2024-11-17 12:25:00|             1|              1|         0|
|2024-11-17 12:30:00|             0|              1|        -1|
|2024-11-17 13:00:00|             0|              1|        -1|
|2024-11-17 15:00:00|             3|              0|         3|
|2024-11-17 15:05:00|             0|    

In [207]:
time_window = Window.orderBy(F.col("time").asc())

In [214]:
schedule_df = schedule_df.withColumn("platforms", F.sum("difference").over(time_window))

In [215]:
schedule_df.show()

25/03/01 17:44:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/01 17:44:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/01 17:44:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/01 17:44:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/01 17:44:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/01 17:45:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/01 1

+-------------------+--------------+---------------+----------+---------+
|               time|trains_arrived|trains_departed|difference|platforms|
+-------------------+--------------+---------------+----------+---------+
|2024-11-17 08:00:00|             1|              0|         1|        1|
|2024-11-17 08:05:00|             2|              0|         2|        3|
|2024-11-17 08:10:00|             2|              1|         1|        4|
|2024-11-17 08:15:00|             0|              1|        -1|        3|
|2024-11-17 08:20:00|             0|              2|        -2|        1|
|2024-11-17 08:25:00|             0|              1|        -1|        0|
|2024-11-17 12:15:00|             1|              0|         1|        1|
|2024-11-17 12:20:00|             1|              0|         1|        2|
|2024-11-17 12:25:00|             1|              1|         0|        2|
|2024-11-17 12:30:00|             0|              1|        -1|        1|
|2024-11-17 13:00:00|             0|  

# Question 6
IBM is working on a new feature to analyze user purchasing behavior for all Fridays in the first quarter of the year. For each Friday separately, calculate the average amount users have spent per order. The output should contain the week number of that Friday and average amount spent.

In [217]:
# Initialize Spark session
spark = SparkSession.builder.appName("UserPurchasesQ1Fridays").getOrCreate()
# Sample data for user purchases
user_purchases_data = [
    (1047, '2023-01-01', 288, 'Sunday'),
    (1099, '2023-01-04', 803, 'Wednesday'),
    (1055, '2023-01-07', 546, 'Saturday'),
    (1040, '2023-01-10', 680, 'Tuesday'),
    (1052, '2023-01-13', 889, 'Friday'),
    (1052, '2023-01-13', 596, 'Friday'),
    (1016, '2023-01-16', 960, 'Monday'),
    (1023, '2023-01-17', 861, 'Tuesday'),
    (1010, '2023-01-19', 758, 'Thursday'),
    (1013, '2023-01-19', 346, 'Thursday'),
    (1069, '2023-01-21', 541, 'Saturday'),
    (1030, '2023-01-22', 175, 'Sunday'),
    (1034, '2023-01-23', 707, 'Monday'),
    (1019, '2023-01-25', 253, 'Wednesday'),
    (1052, '2023-01-25', 868, 'Wednesday'),
    (1095, '2023-01-27', 424, 'Friday'),
    (1017, '2023-01-28', 755, 'Saturday'),
    (1010, '2023-01-29', 615, 'Sunday'),
    (1063, '2023-01-31', 534, 'Tuesday'),
    (1019, '2023-02-03', 185, 'Friday'),
    (1019, '2023-02-03', 995, 'Friday'),
    (1092, '2023-02-06', 796, 'Monday'),
    (1058, '2023-02-09', 384, 'Thursday'),
    (1055, '2023-02-12', 319, 'Sunday'),
    (1090, '2023-02-15', 168, 'Wednesday'),
    (1090, '2023-02-18', 146, 'Saturday'),
    (1062, '2023-02-21', 193, 'Tuesday'),
    (1023, '2023-02-24', 259, 'Friday')
]
# Define schema for user purchases
columns_user_purchases = ["user_id", "date", "amount_spent", "day_name"]
# Create DataFrame for user purchases
user_purchases_df = spark.createDataFrame(user_purchases_data, columns_user_purchases)

In [218]:
# Filter the data for Fridays in the first quarter (Q1) of the year
df_fridays_q1 = user_purchases_df.filter(
    (F.col("day_name") == "Friday") & 
    (F.col("date").between("2023-01-01", "2023-03-31"))
)

df_fridays_q1.show()

+-------+----------+------------+--------+
|user_id|      date|amount_spent|day_name|
+-------+----------+------------+--------+
|   1052|2023-01-13|         889|  Friday|
|   1052|2023-01-13|         596|  Friday|
|   1095|2023-01-27|         424|  Friday|
|   1019|2023-02-03|         185|  Friday|
|   1019|2023-02-03|         995|  Friday|
|   1023|2023-02-24|         259|  Friday|
+-------+----------+------------+--------+



In [245]:
from datetime import datetime
from pyspark.sql import types

@F.udf(returnType=types.StringType())
def get_week_number(date):
    d = datetime.strptime(date, "%Y-%M-%d")
    return d.isocalendar().week

In [246]:
df_fridays_q1_groupped = df_fridays_q1.groupBy("date").agg(F.avg("amount_spent").alias("avg_ammount_spent"))

In [248]:
df_fridays_q1_groupped = df_fridays_q1_groupped.withColumn("week_number", F.weekofyear(F.col("date")))
df_fridays_q1_groupped.show()

+----------+-----------------+-----------+
|      date|avg_ammount_spent|week_number|
+----------+-----------------+-----------+
|2023-01-13|            742.5|          2|
|2023-01-27|            424.0|          4|
|2023-02-03|            590.0|          5|
|2023-02-24|            259.0|          8|
+----------+-----------------+-----------+



# Question 7
Find the number of Apple product users (MacBook-Pro, iPhone 5s, iPad-air) and the total number of users with any device, grouped by language. Output the language along with the total number of Apple users and users with any device. Order the results by the number of total users in descending order.

In [249]:
# Initialize Spark session
spark = SparkSession.builder.appName("AppleUsers").getOrCreate()
# Sample data for playbook_users
users_data = [
    (1, '2024-01-01 08:00:00', 101, 'English', '2024-01-05 10:00:00', 'Active'),
    (2, '2024-01-02 09:00:00', 102, 'Spanish', '2024-01-06 11:00:00', 'Inactive'),
    (3, '2024-01-03 10:00:00', 103, 'French', '2024-01-07 12:00:00', 'Active'),
    (4, '2024-01-04 11:00:00', 104, 'English', '2024-01-08 13:00:00', 'Active'),
    (5, '2024-01-05 12:00:00', 105, 'Spanish', '2024-01-09 14:00:00', 'Inactive')
]
# Sample data for playbook_events
events_data = [
    (1, '2024-01-05 14:00:00', 'Click', 'Login', 'USA', 'MacBook-Pro'),
    (2, '2024-01-06 15:00:00', 'View', 'Dashboard', 'Spain', 'iPhone 5s'),
    (3, '2024-01-07 16:00:00', 'Click', 'Logout', 'France', 'iPad-air'),
    (4, '2024-01-08 17:00:00', 'Purchase', 'Subscription', 'USA', 'Windows-Laptop'),
    (5, '2024-01-09 18:00:00', 'Click', 'Login', 'Spain', 'Android-Phone')
]
# Define schema for users and events data
users_columns = [
  "user_id", 
  "created_at", 
  "company_id", 
  "language", 
  "activated_at", 
  "state"
]
events_columns = [
  "user_id", 
  "occurred_at", 
  "event_type", 
  "event_name", 
  "location", 
  "device"
]
# Create DataFrames
users_df = spark.createDataFrame(users_data, users_columns)
events_df = spark.createDataFrame(events_data, events_columns)

25/03/01 20:36:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [250]:
users_df.show()

+-------+-------------------+----------+--------+-------------------+--------+
|user_id|         created_at|company_id|language|       activated_at|   state|
+-------+-------------------+----------+--------+-------------------+--------+
|      1|2024-01-01 08:00:00|       101| English|2024-01-05 10:00:00|  Active|
|      2|2024-01-02 09:00:00|       102| Spanish|2024-01-06 11:00:00|Inactive|
|      3|2024-01-03 10:00:00|       103|  French|2024-01-07 12:00:00|  Active|
|      4|2024-01-04 11:00:00|       104| English|2024-01-08 13:00:00|  Active|
|      5|2024-01-05 12:00:00|       105| Spanish|2024-01-09 14:00:00|Inactive|
+-------+-------------------+----------+--------+-------------------+--------+



In [262]:
user_events_df = events_filtered_df.join(users_df.select("user_id", "language"), 'user_id', 'inner').select(events_filtered_df.user_id, "device", "language")
user_events_df.show()

+-------+--------------+--------+
|user_id|        device|language|
+-------+--------------+--------+
|      1|   MacBook-Pro| English|
|      2|     iPhone 5s| Spanish|
|      3|      iPad-air|  French|
|      5| Android-Phone| Spanish|
|      4|Windows-Laptop| English|
+-------+--------------+--------+



In [277]:
apple_devices = user_events_df.filter((F.col("device").like("Mac%")) | F.col("device").like("iP%"))

In [278]:
apple_devices_groupped = apple_devices.groupBy("language").agg(F.count_distinct("user_id").alias("apple_users"))

In [279]:
total_devices = user_events_df.groupBy("language").agg(F.count_distinct("user_id").alias("total_users"))

In [280]:
languages = apple_devices_groupped.join(total_devices, "language", "outer")

In [281]:
languages.show()

+--------+-----------+-----------+
|language|apple_users|total_users|
+--------+-----------+-----------+
| English|          1|          2|
|  French|          1|          1|
| Spanish|          1|          2|
+--------+-----------+-----------+



# Question 8
We have two datasets:

Sessions Table: Contains records of when users started their sessions.

Order Summary Table: Contains records of orders placed by users along with their values.

We want to:

Find users who started a session and placed an order on the same day.

Calculate the total number of orders and the total order value for those users.

In [282]:
# Initialize Spark session
spark = SparkSession.builder.appName("SessionOrderAnalysis").getOrCreate()
# Sample data for sessions
sessions_data = [
    (1, 1, '2024-01-01 00:00:00'),
    (2, 2, '2024-01-02 00:00:00'),
    (3, 3, '2024-01-05 00:00:00'),
    (4, 3, '2024-01-05 00:00:00'),
    (5, 4, '2024-01-03 00:00:00'),
    (6, 4, '2024-01-03 00:00:00'),
    (7, 5, '2024-01-04 00:00:00'),
    (8, 5, '2024-01-04 00:00:00'),
    (9, 3, '2024-01-05 00:00:00'),
    (10, 5, '2024-01-04 00:00:00')
]
# Sample data for orders
orders_data = [
    (1, 1, 152, '2024-01-01 00:00:00'),
    (2, 2, 485, '2024-01-02 00:00:00'),
    (3, 3, 398, '2024-01-05 00:00:00'),
    (4, 3, 320, '2024-01-05 00:00:00'),
    (5, 4, 156, '2024-01-03 00:00:00'),
    (6, 4, 121, '2024-01-03 00:00:00'),
    (7, 5, 238, '2024-01-04 00:00:00'),
    (8, 5, 70, '2024-01-04 00:00:00'),
    (9, 3, 152, '2024-01-05 00:00:00'),
    (10, 5, 171, '2024-01-04 00:00:00')
]
session_columns = ["session_id", "user_id", "session_date"]
orders_columns = ["order_id", "user_id", "order_value", "order_date"]
# Convert data into DataFrames
sessions_df = spark.createDataFrame(sessions_data, session_columns)
orders_df = spark.createDataFrame(orders_data, orders_columns)

25/03/02 13:13:09 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [283]:
sessions_df.show()

+----------+-------+-------------------+
|session_id|user_id|       session_date|
+----------+-------+-------------------+
|         1|      1|2024-01-01 00:00:00|
|         2|      2|2024-01-02 00:00:00|
|         3|      3|2024-01-05 00:00:00|
|         4|      3|2024-01-05 00:00:00|
|         5|      4|2024-01-03 00:00:00|
|         6|      4|2024-01-03 00:00:00|
|         7|      5|2024-01-04 00:00:00|
|         8|      5|2024-01-04 00:00:00|
|         9|      3|2024-01-05 00:00:00|
|        10|      5|2024-01-04 00:00:00|
+----------+-------+-------------------+



In [284]:
orders_df.show()

+--------+-------+-----------+-------------------+
|order_id|user_id|order_value|         order_date|
+--------+-------+-----------+-------------------+
|       1|      1|        152|2024-01-01 00:00:00|
|       2|      2|        485|2024-01-02 00:00:00|
|       3|      3|        398|2024-01-05 00:00:00|
|       4|      3|        320|2024-01-05 00:00:00|
|       5|      4|        156|2024-01-03 00:00:00|
|       6|      4|        121|2024-01-03 00:00:00|
|       7|      5|        238|2024-01-04 00:00:00|
|       8|      5|         70|2024-01-04 00:00:00|
|       9|      3|        152|2024-01-05 00:00:00|
|      10|      5|        171|2024-01-04 00:00:00|
+--------+-------+-----------+-------------------+



In [296]:
sessions_df = sessions_df.withColumn("session_date", F.to_date(F.col("session_date"))).drop_duplicates(['user_id', 'session_date'])
orders_df = orders_df.withColumn("order_date", F.to_date(F.col("order_date")))
# Join the DataFrames
sessions_orders_df = sessions_df.join(orders_df, "user_id", "inner")

In [297]:
sessions_orders_df = sessions_orders_df.filter(F.col("session_date") == F.col("order_date"))

In [298]:
sessions_orders_df.show()

+-------+----------+------------+--------+-----------+----------+
|user_id|session_id|session_date|order_id|order_value|order_date|
+-------+----------+------------+--------+-----------+----------+
|      2|         2|  2024-01-02|       2|        485|2024-01-02|
|      1|         1|  2024-01-01|       1|        152|2024-01-01|
|      3|         3|  2024-01-05|       3|        398|2024-01-05|
|      3|         3|  2024-01-05|       4|        320|2024-01-05|
|      4|         5|  2024-01-03|       5|        156|2024-01-03|
|      4|         5|  2024-01-03|       6|        121|2024-01-03|
|      3|         3|  2024-01-05|       9|        152|2024-01-05|
|      5|         7|  2024-01-04|       7|        238|2024-01-04|
|      5|         7|  2024-01-04|       8|         70|2024-01-04|
|      5|         7|  2024-01-04|      10|        171|2024-01-04|
+-------+----------+------------+--------+-----------+----------+



In [299]:
sessions_orders_df.groupBy("user_id", "session_date").agg(F.sum("order_value").alias("total_order_value"),
                                          F.count("order_id").alias("total_number_of_orders")).show()

+-------+------------+-----------------+----------------------+
|user_id|session_date|total_order_value|total_number_of_orders|
+-------+------------+-----------------+----------------------+
|      2|  2024-01-02|              485|                     1|
|      1|  2024-01-01|              152|                     1|
|      3|  2024-01-05|              870|                     3|
|      4|  2024-01-03|              277|                     2|
|      5|  2024-01-04|              479|                     3|
+-------+------------+-----------------+----------------------+



# Question 9
We are given a table called customer_state_log containing the following columns:

- cust_id
- state: The state of the session, where 1 indicates the session is active and 0 indicates the session has ended.
- timestamp: The timestamp when the state change occurred.

Our task is to calculate how many hours each user was active during the day based on the state transitions.

In [3]:
# Initialize Spark session
spark = SparkSession.builder.master("local").appName("Customer Session Hours").getOrCreate()
# Sample data (as given in the problem)
data = [
    ('c001', 1, '07:00:00'),
    ('c001', 0, '09:30:00'),
    ('c001', 1, '12:00:00'),
    ('c001', 0, '14:30:00'),
    ('c002', 1, '08:00:00'),
    ('c002', 0, '09:30:00'),
    ('c002', 1, '11:00:00'),
    ('c002', 0, '12:30:00'),
    ('c002', 1, '15:00:00'),
    ('c002', 0, '16:30:00'),
    ('c003', 1, '09:00:00'),
    ('c003', 0, '10:30:00'),
    ('c004', 1, '10:00:00'),
    ('c004', 0, '10:30:00'),
    ('c004', 1, '14:00:00'),
    ('c004', 0, '15:30:00'),
    ('c005', 1, '10:00:00'),
    ('c005', 0, '14:30:00'),
    ('c005', 1, '15:30:00'),
    ('c005', 0, '18:30:00')
]
# Create a DataFrame
columns = ["cust_id", "state", "timestamp"]
df = spark.createDataFrame(data, columns)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/04 13:22:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
user_sessions = df.withColumn("timestamp", F.to_unix_timestamp(F.to_timestamp((F.col("timestamp")))))

In [9]:
user_sessions.show()

+-------+-----+----------+
|cust_id|state| timestamp|
+-------+-----+----------+
|   c001|    1|1741064400|
|   c001|    0|1741073400|
|   c001|    1|1741082400|
|   c001|    0|1741091400|
|   c002|    1|1741068000|
|   c002|    0|1741073400|
|   c002|    1|1741078800|
|   c002|    0|1741084200|
|   c002|    1|1741093200|
|   c002|    0|1741098600|
|   c003|    1|1741071600|
|   c003|    0|1741077000|
|   c004|    1|1741075200|
|   c004|    0|1741077000|
|   c004|    1|1741089600|
|   c004|    0|1741095000|
|   c005|    1|1741075200|
|   c005|    0|1741091400|
|   c005|    1|1741095000|
|   c005|    0|1741105800|
+-------+-----+----------+



In [10]:
window_spec = Window.partitionBy('cust_id').orderBy(F.col('timestamp').asc())

In [15]:
user_sessions = user_sessions.withColumn('session_duration', (F.lead('timestamp').over(window_spec) - F.col('timestamp')) / 3600)

In [17]:
user_sessions.filter(F.col("state") == 1).groupBy("cust_id").agg(F.sum("session_duration").alias("total_session_hours")).show()

+-------+-------------------+
|cust_id|total_session_hours|
+-------+-------------------+
|   c001|                5.0|
|   c002|                4.5|
|   c003|                1.5|
|   c004|                2.0|
|   c005|                7.5|
+-------+-------------------+



# Question 10
We have to find the 3rd highest total transaction amount from the records. 

We have two tables: one containing customer details (customers) and the other storing transaction data (card_orders).

Our goal is to retrieve the customer who ranks third in terms of total transaction amount.

In [19]:
# Initialize the Spark session
spark = SparkSession.builder.master("local").appName("CustomerTransactions").getOrCreate()
# Create the customers DataFrame
customers_data = [
    (1, 'Jill', 'Doe', 'New York', '123 Main St', '555-1234'),
    (2, 'Henry', 'Smith', 'Los Angeles', '456 Oak Ave', '555-5678'),
    (3, 'William', 'Johnson', 'Chicago', '789 Pine Rd', '555-8765'),
    (4, 'Emma', 'Daniel', 'Houston', '321 Maple Dr', '555-4321'),
    (5, 'Charlie', 'Davis', 'Phoenix', '654 Elm St', '555-6789')
]
customers_columns = [
  'id', 
  'first_name', 
  'last_name', 
  'city', 
  'address', 
  'phone_number'
]
customers_df = spark.createDataFrame(customers_data, customers_columns)

card_orders_data = [
    (1, 1, '2024-11-01 10:00:00', 'Electronics', 200),
    (2, 2, '2024-11-02 11:30:00', 'Groceries', 150),
    (3, 1, '2024-11-03 15:45:00', 'Clothing', 120),
    (4, 3, '2024-11-04 09:10:00', 'Books', 90),
    (8, 3, '2024-11-08 10:20:00', 'Groceries', 130),
    (9, 1, '2024-11-09 12:00:00', 'Books', 180),
    (10, 4, '2024-11-10 11:15:00', 'Electronics', 200),
    (11, 5, '2024-11-11 14:45:00', 'Furniture', 150),
    (12, 2, '2024-11-12 09:30:00', 'Furniture', 180)
]
card_orders_columns = [
  'order_id', 
  'cust_id', 
  'order_date', 
  'order_details', 
  'total_order_cost'
]
card_orders_df = spark.createDataFrame(card_orders_data, card_orders_columns)

In [20]:
customers_df.show()

+---+----------+---------+-----------+------------+------------+
| id|first_name|last_name|       city|     address|phone_number|
+---+----------+---------+-----------+------------+------------+
|  1|      Jill|      Doe|   New York| 123 Main St|    555-1234|
|  2|     Henry|    Smith|Los Angeles| 456 Oak Ave|    555-5678|
|  3|   William|  Johnson|    Chicago| 789 Pine Rd|    555-8765|
|  4|      Emma|   Daniel|    Houston|321 Maple Dr|    555-4321|
|  5|   Charlie|    Davis|    Phoenix|  654 Elm St|    555-6789|
+---+----------+---------+-----------+------------+------------+



In [21]:
card_orders_df.show()

+--------+-------+-------------------+-------------+----------------+
|order_id|cust_id|         order_date|order_details|total_order_cost|
+--------+-------+-------------------+-------------+----------------+
|       1|      1|2024-11-01 10:00:00|  Electronics|             200|
|       2|      2|2024-11-02 11:30:00|    Groceries|             150|
|       3|      1|2024-11-03 15:45:00|     Clothing|             120|
|       4|      3|2024-11-04 09:10:00|        Books|              90|
|       8|      3|2024-11-08 10:20:00|    Groceries|             130|
|       9|      1|2024-11-09 12:00:00|        Books|             180|
|      10|      4|2024-11-10 11:15:00|  Electronics|             200|
|      11|      5|2024-11-11 14:45:00|    Furniture|             150|
|      12|      2|2024-11-12 09:30:00|    Furniture|             180|
+--------+-------+-------------------+-------------+----------------+



In [23]:
total_costs_df = card_orders_df.groupby("cust_id").agg(F.sum("total_order_cost").alias("total_orders_cost"))

In [30]:
total_costs_cust_df = customers_df.join(total_costs_df, F.col("cust_id") == F.col("id"), "inner").orderBy(F.col("total_orders_cost").desc())

In [31]:
window_spec = Window.orderBy(F.col("total_orders_cost").desc())

In [33]:
total_costs_cust_df.withColumn("rank", F.dense_rank().over(window_spec)).filter(F.col("rank") == 3).show()

25/03/04 13:53:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/04 13:53:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/04 13:53:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/04 13:53:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/04 13:53:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/04 13:53:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/04 1

+---+----------+---------+-------+-----------+------------+-------+-----------------+----+
| id|first_name|last_name|   city|    address|phone_number|cust_id|total_orders_cost|rank|
+---+----------+---------+-------+-----------+------------+-------+-----------------+----+
|  3|   William|  Johnson|Chicago|789 Pine Rd|    555-8765|      3|              220|   3|
+---+----------+---------+-------+-----------+------------+-------+-----------------+----+



25/03/05 00:09:16 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 7197886 ms exceeds timeout 120000 ms
25/03/05 00:09:16 WARN SparkContext: Killing executors is not supported by current scheduler.
25/03/05 02:09:19 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 